In [1]:

import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [2]:
# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [3]:
# Load pre-trained VGG16 model with weights trained on ImageNet
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


In [4]:
# Freeze all convolutional layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Add custom classification layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)


In [6]:
# Define the complete model
model = Model(inputs=base_model.input, outputs=output)


In [7]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the custom classifier layers
history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))


Epoch 1/5
352/391 ━━━━━━━━━━━━━━━━━━━━ 11s 307ms/step - accuracy: 0.3251 - loss: 1.9009

KeyboardInterrupt: 

In [ ]:
# Unfreeze last 4 layers for fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

In [ ]:
# Recompile the model after unfreezing
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model with a lower learning rate
model.fit(x_train, y_train, epochs=5, batch_size=256, validation_data=(x_test, y_test))

In [ ]:
# Evaluate the model
t_loss, t_acc = model.evaluate(x_test, y_test)
print(f"Test Loss: {t_loss:.4f}")
print(f"Test Accuracy: {t_acc:.4f}")